In [1]:
import os
import psycopg2
import pandas as pd
import json

## Note:

1. The **URL** to the google drive containing the **dec-jan_flights.csv** file: https://drive.google.com/drive/folders/1m61S5WMcSpdXV6pq3k7srXOQIr0fnlOv?usp=sharing
    - The code to create this file is below.


In [2]:
def connect_db():
    '''
    Creating connection to DB
    '''
    try:
        conn = psycopg2.connect(
                host=os.environ['MID_PROJ_DB_HOST'],
                database=os.environ['MID_PROJ_DB'],
                user=os.environ['MID_PROJ_DB_USERNAME'],
                password=os.environ['MID_PROJ_DB_PASSWORD'],
                port='5432')
        print('Db Connected succsefuly')
        return conn
    except (Exception, Error) as error:
        print("Error while connecting")

In [3]:
def query_db(query,conn):
    cursor = conn.cursor()
    cursor.execute(query)
    return {'data': cursor.fetchall(), 'columns': [desc[0].strip() for desc in cursor.description]} 

In [4]:
def write_to_file(csv_name,query_output):
    df = pd.DataFrame(query_output['data'])
    df.columns = query_output['columns']
    df.to_csv(csv_name + '.csv', sep=",", index = False)

In [5]:
conn = connect_db()

Db Connected succsefuly


In [19]:
# # Note: this is one method of data sampling, we may want to explore others
# # This method primarily focuses on looking at the 2 week time period just before when we want to predict, over a yearly basis

flights_jan_dec_19_query="""select * from flights
where fl_date > '2018-12-31' 
and fl_date < '2019-02-01' 
and origin_city_name in (select origin_city_name
from flights
group by origin_city_name
order by COUNT(origin_city_name) desc 
limit 10)
and dest_city_name in (select dest_city_name
from flights
group by dest_city_name
order by COUNT(dest_city_name) desc 
limit 10)
union 
select * from flights 
where  fl_date > '2019-11-30' 
and fl_date < '2020-01-01'
and origin_city_name in (select origin_city_name
from flights
group by origin_city_name
order by COUNT(origin_city_name) desc 
limit 10)
and dest_city_name in (select dest_city_name
from flights
group by dest_city_name
order by COUNT(dest_city_name) desc 
limit 10);
"""
flights_jan_dec_18_query="""select * from flights
where fl_date > '2017-12-31' 
and fl_date < '2018-02-01' 
and origin_city_name in (select origin_city_name
from flights
group by origin_city_name
order by COUNT(origin_city_name) desc 
limit 10)
and dest_city_name in (select dest_city_name
from flights
group by dest_city_name
order by COUNT(dest_city_name) desc 
limit 10)
union 
select * from flights 
where  fl_date > '2018-11-30' 
and fl_date < '2019-01-01'
and origin_city_name in (select origin_city_name
from flights
group by origin_city_name
order by COUNT(origin_city_name) desc 
limit 10)
and dest_city_name in (select dest_city_name
from flights
group by dest_city_name
order by COUNT(dest_city_name) desc 
limit 10);
"""

### NOTE: Elena handled the flights + weather, code below may not work correctly.

In [ ]:
#flights_jan_dec_19 = query_db(flights_jan_dec_19_query,conn)
#flights_jan_dec_18 = query_db(flights_jan_dec_18_query,conn)

In [ ]:
#write_to_file('flights_jan_dec_19',flights_jan_dec_19)
#write_to_file('flights_jan_dec_18',flights_jan_dec_18)

In [ ]:
#flights_19 = pd.read_csv('flights_jan_dec_19.csv')
#flights_18 = pd.read_csv('flights_jan_dec_18.csv')

In [9]:
# flights = [flights_18, flights_19]
# flights = pd.concat(flights)

In [11]:
# flights.head()

In [12]:
# flights.to_csv('dec-jan_flights.csv')

# Passenger data

Obtain passenger data for the top 10 cities:
- Chicago, IL
- Atlanta, GA
- Dallas/Fort Worth, TX
- Denver, CO
- Los Angeles, CA
- Charlotte, NC
- New York, NY
- Houston, TX
- Washington, DC
- Seattle, WA

**Note**: Passenger information is **ROUTE SPECIFIC** for the **ENTIRE MONTH**

In [13]:
passengers_jan_18_query="""
select * from passengers
where origin_city_name IN ('Chicago, IL','Atlanta, GA','Dallas/Fort Worth, TX','Denver, CO','Los Angeles, CA','Charlotte, NC','New York, NY','Houston, TX','Washington, DC','Seattle, WA')
and dest_city_name IN ('Chicago, IL','Atlanta, GA','Dallas/Fort Worth, TX','Denver, CO','Los Angeles, CA','Charlotte, NC','New York, NY','Houston, TX','Washington, DC','Seattle, WA')
and month = 1
and year = 2018
"""

passengers_jan_18 = query_db(passengers_jan_18_query,conn)

In [14]:
passengers_dec_18_query="""
select * from passengers
where origin_city_name IN ('Chicago, IL','Atlanta, GA','Dallas/Fort Worth, TX','Denver, CO','Los Angeles, CA','Charlotte, NC','New York, NY','Houston, TX','Washington, DC','Seattle, WA')
and dest_city_name IN ('Chicago, IL','Atlanta, GA','Dallas/Fort Worth, TX','Denver, CO','Los Angeles, CA','Charlotte, NC','New York, NY','Houston, TX','Washington, DC','Seattle, WA')
and month = 12
and year = 2018
"""

passengers_dec_18 = query_db(passengers_dec_18_query,conn)

In [15]:
passengers_jan_19_query="""
select * from passengers
where origin_city_name IN ('Chicago, IL','Atlanta, GA','Dallas/Fort Worth, TX','Denver, CO','Los Angeles, CA','Charlotte, NC','New York, NY','Houston, TX','Washington, DC','Seattle, WA')
and dest_city_name IN ('Chicago, IL','Atlanta, GA','Dallas/Fort Worth, TX','Denver, CO','Los Angeles, CA','Charlotte, NC','New York, NY','Houston, TX','Washington, DC','Seattle, WA')
and month = 1
and year = 2019
"""

passengers_jan_19 = query_db(passengers_jan_19_query,conn)

In [16]:
passengers_dec_19_query="""
select * from passengers
where origin_city_name IN ('Chicago, IL','Atlanta, GA','Dallas/Fort Worth, TX','Denver, CO','Los Angeles, CA','Charlotte, NC','New York, NY','Houston, TX','Washington, DC','Seattle, WA')
and dest_city_name IN ('Chicago, IL','Atlanta, GA','Dallas/Fort Worth, TX','Denver, CO','Los Angeles, CA','Charlotte, NC','New York, NY','Houston, TX','Washington, DC','Seattle, WA')
and month = 12
and year = 2019
"""

passengers_dec_19 = query_db(passengers_dec_19_query,conn)

In [17]:
# Write to files
# write_to_file('passengers_jan_18',passengers_jan_18)
# write_to_file('passengers_dec_18',passengers_dec_18)
# write_to_file('passengers_jan_19',passengers_jan_19)
# write_to_file('passengers_dec_19',passengers_dec_19)

# Note: Fix the PATH to open these

In [89]:
passengers_jan_18 = pd.read_csv('data_archive\\passengers_jan_18.csv', sep = ',')
passengers_dec_18 = pd.read_csv('data_archive\\passengers_dec_18.csv', sep = ',')
passengers_jan_19 = pd.read_csv('data_archive\\passengers_jan_19.csv', sep = ',')
passengers_dec_19 = pd.read_csv('data_archive\\passengers_dec_19.csv', sep = ',')

### Decide which columns we want to carry over to our flights database
**Note:** It's probably best to create a separate .csv files, so we have our original flights data as a backup.

In [23]:
passengers_jan_18.columns

Index(['departures_scheduled', 'departures_performed', 'payload', 'seats',
       'passengers', 'freight', 'mail', 'distance', 'ramp_to_ramp', 'air_time',
       'unique_carrier', 'airline_id', 'unique_carrier_name', 'region',
       'carrier', 'carrier_name', 'carrier_group', 'carrier_group_new',
       'origin_airport_id', 'origin_city_market_id', 'origin',
       'origin_city_name', 'origin_country', 'origin_country_name',
       'dest_airport_id', 'dest_city_market_id', 'dest', 'dest_city_name',
       'dest_country', 'dest_country_name', 'aircraft_group', 'aircraft_type',
       'aircraft_config', 'year', 'month', 'distance_group', 'class',
       'data_source'],
      dtype='object')

In [54]:
key_features = ['departures_scheduled', 'departures_performed', 'seats', 'passengers', 
                'freight', 'mail', 
                'distance', 'ramp_to_ramp', 'air_time', 
                'unique_carrier', 'unique_carrier_name', 'aircraft_type', 
                'origin_airport_id', 'origin_city_market_id', 'origin_city_name', 
                'dest_airport_id', 'dest_city_market_id', 'dest_city_name',
                'year', 'month'
               ]

In [57]:
passengers_jan_18.shape

(1398, 38)

In [78]:
passengers_jan_18_filtered = passengers_jan_18[key_features]
passengers_dec_18_filtered = passengers_dec_18[key_features]
passengers_jan_19_filtered = passengers_jan_19[key_features]
passengers_dec_19_filtered = passengers_dec_19[key_features]

In [90]:
passengers = [passengers_jan_18, passengers_dec_18, passengers_jan_19, passengers_dec_19]
passengers = pd.concat(passengers)

In [91]:
passengers.to_csv('passengers_jan-dec_18-19_raw.csv')

In [84]:
passengers_filtered = [passengers_jan_18_filtered, passengers_dec_18_filtered, passengers_jan_19_filtered, passengers_dec_19_filtered]
passengers_filtered = pd.concat(passengers_filtered)

In [88]:
passengers_filtered.to_csv('passengers_jan-dec_18-19.csv')

In [86]:
passengers_filtered.shape # we removed 18 columns

(6094, 20)

In [87]:
# note how this carrier never has a flight scheduled
passengers_filtered[passengers_filtered['unique_carrier_name']=='AirBridgeCargo Airlines Limited']

,departures_scheduled,departures_performed,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,unique_carrier,unique_carrier_name,aircraft_type,origin_airport_id,origin_city_market_id,origin_city_name,dest_airport_id,dest_city_market_id,dest_city_name,year,month
475,0.0,6.0,0.0,0.0,369118.0,0.0,1947.0,0.0,0.0,RU,AirBridgeCargo Airlines Limited,820,12892,32575,"Los Angeles, CA",10397,30397,"Atlanta, GA",2018,1
476,0.0,20.0,0.0,0.0,1486456.0,0.0,1947.0,0.0,0.0,RU,AirBridgeCargo Airlines Limited,821,12892,32575,"Los Angeles, CA",10397,30397,"Atlanta, GA",2018,1
477,0.0,26.0,0.0,0.0,2082782.0,0.0,1744.0,0.0,0.0,RU,AirBridgeCargo Airlines Limited,821,12892,32575,"Los Angeles, CA",13930,30977,"Chicago, IL",2018,1
478,0.0,7.0,0.0,0.0,530985.0,0.0,802.0,0.0,0.0,RU,AirBridgeCargo Airlines Limited,820,13930,30977,"Chicago, IL",11298,30194,"Dallas/Fort Worth, TX",2018,1
479,0.0,19.0,0.0,0.0,1543648.0,0.0,802.0,0.0,0.0,RU,AirBridgeCargo Airlines Limited,821,13930,30977,"Chicago, IL",11298,30194,"Dallas/Fort Worth, TX",2018,1
456,0.0,16.0,0.0,0.0,1603699.0,0.0,1947.0,0.0,0.0,RU,AirBridgeCargo Airlines Limited,821,12892,32575,"Los Angeles, CA",10397,30397,"Atlanta, GA",2018,12
457,0.0,25.0,0.0,0.0,2334840.0,0.0,1744.0,0.0,0.0,RU,AirBridgeCargo Airlines Limited,821,12892,32575,"Los Angeles, CA",13930,30977,"Chicago, IL",2018,12
458,0.0,4.0,0.0,0.0,232386.0,0.0,801.0,0.0,0.0,RU,AirBridgeCargo Airlines Limited,820,13930,30977,"Chicago, IL",11298,30194,"Dallas/Fort Worth, TX",2018,12
459,0.0,10.0,0.0,0.0,552068.0,0.0,801.0,0.0,0.0,RU,AirBridgeCargo Airlines Limited,821,13930,30977,"Chicago, IL",11298,30194,"Dallas/Fort Worth, TX",2018,12
73,0.0,6.0,0.0,0.0,454590.0,0.0,1947.0,0.0,0.0,RU,AirBridgeCargo Airlines Limited,820,12892,32575,"Los Angeles, CA",10397,30397,"Atlanta, GA",2019,1
